In [1]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras import layers, models

import numpy as np
import pickle

In [2]:
(train_data, train_labels), (test_data, test_labels) = pickle.load( open( "imdb.p", "rb" ) )
print("train data shape: ", train_data.shape)
print("train labels shape: ", train_labels.shape)

train data shape:  (25000,)
train labels shape:  (25000,)


In [3]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(y, x) for x,y in word_index.items()])

decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])
print(decoded_review)

? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you thi

In [4]:
def vectorize_sequences(sequences, maxlen = 10000):
    out = np.zeros((len(sequences), maxlen))
    for n, i in enumerate(sequences):
        out[n, i] = 1.
    return out


x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)
print(x_train[0])

y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

[0. 1. 1. ... 0. 0. 0.]


In [5]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape = (10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer = 'rmsprop', metrics = ['acc'], loss = 'binary_crossentropy')
model.fit(x_train, y_train, epochs=4, batch_size = 512)#, validation_split = 0.2)

Epoch 1/4
25000/25000 [==============================] - 2s 90us/sample - loss: 0.4609 - acc: 0.8238
Epoch 2/4
25000/25000 [==============================] - 2s 69us/sample - loss: 0.2631 - acc: 0.9093
Epoch 3/4
25000/25000 [==============================] - 2s 69us/sample - loss: 0.2026 - acc: 0.9285
Epoch 4/4
25000/25000 [==============================] - 2s 67us/sample - loss: 0.1702 - acc: 0.9394


In [6]:
model.evaluate(x_test, y_test)

25000/25000 [==============================] - 3s 106us/sample - loss: 0.2902 - acc: 0.8867


[0.2902216139125824, 0.88668]

In [7]:
test_predictions = model.predict(x_test)
print(test_predictions[0])

[0.20560718]


In [8]:
#top cases when the model is not sure about choosing class (closest to 0.5)
differences = np.absolute(test_predictions - 0.5)
top_20_uncertain = np.argsort(differences, axis = 0)[:20]

In [9]:
for item_num in top_20_uncertain:
    itn = item_num[0]
    decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in test_data[itn]])
    print("Item {}, probability {}, label {}".format(item_num[0], model.predict(x_test[np.newaxis, itn])[0], test_labels[itn]))
    print(decoded_review + '\n'*3)

Item 6125, probability [0.4999049], label 0
? another example that we should stay away from trying to do spectacular action movies in sweden it ? work except for still ? ? does the best he can i suppose and some scenes are mildly effective but the plot is full of holes why does hamilton continue the attack on the base knowing that his wife is held hostage it was fun to see mark ? but his bad guy part was very ? ? constantly amazed that relatively big swedish movies like this get made without a sensible ? script 1 out of 5



Item 3627, probability [0.5001886], label 0
? stranded in space 1972 mst3k version a very not good tv movie pilot for a never to be made series in which an astronaut finds himself trapped on ? evil twin having a planet of identical size and mass ? in the same plane as the earth but on the opposite side of the sun is a well worn sf ? the idea is over 2 000 years old having been invented by the ancient ? in this version the counter world is run as an ? ? society wher